In [39]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus

In [46]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'

def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

In [49]:
def fetch_data():
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
              SELECT E.*, F.NRO_DOCUMENTO, CASE WHEN F.ID_TIPO_PERSONA = 2 THEN F.RAZON_SOCIAL ELSE F.NOMBRES + ' ' + F.APELLIDOS END AS RAZON_SOCIAL
                FROM (
                    SELECT doc.ASUNTO as ASUNTO2, doc.ID_CLASE_DOCUMENTO_INTERNO as CLASE2, A.*, CASE WHEN B.SIGLAS_SIAD IS NULL THEN B.SIGLAS ELSE B.SIGLAS_SIAD END AS DEPENDENCIA_ORIGEN,
                            CASE WHEN D.SIGLAS_SIAD IS NULL THEN D.SIGLAS ELSE D.SIGLAS_SIAD END AS DEPENDENCIA_DESTINO, 
                            C.ASUNTO, C.ID_CLASE_DOCUMENTO_INTERNO, C.ID_PERSONA, C.HOJA_TRAMITE, C.ID_ESTADO_DOCUMENTO
                        FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO A
                        LEFT JOIN DB_GENERAL.JCARDENAS.H_DEPENDENCIA B ON A.ID_DEPENDENCIA_ORIGEN = B.CODIGO_DEPENDENCIA
                        LEFT JOIN DB_GENERAL.JCARDENAS.H_DEPENDENCIA D ON A.ID_DEPENDENCIA_DESTINO = D.CODIGO_DEPENDENCIA
                        LEFT JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO C ON A.ID_DOCUMENTO = C.ID_DOCUMENTO
                        LEFT JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO doc ON A.ID_OFICIO = doc.ID_DOCUMENTO
                    ) E
                LEFT JOIN DB_GENERAL.dbo.PERSONA F ON  E.ID_PERSONA = F.ID
        """
        result = pd.read_sql(query, conn)
    finally:
        conn.close()  # Cerrar la conexión al final
    return result

In [50]:
MOVIMIENTO_DOCUMENTO = fetch_data()

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_14724\2175631031.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
